In [6]:
import torch
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-23 16:11:46 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-23 16:11:46 [__init__.py:239] Automatically detected platform cuda.


In [7]:
max_seq_length = 2048  # Can increase for longer reasoning traces
lora_rank = 32         # Larger rank = smarter, but slower

# Load model + tokenizer with vLLM acceleration
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B",
    max_seq_length = max_seq_length,
    load_in_4bit = False,       # False for LoRA 16bit
    fast_inference = True,      # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Quadro T1000 with Max-Q Design. Num GPUs = 1. Max memory: 4.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-0.6B with actual GPU utilization = 55.83%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 4.0 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 1.25 GB. Also swap space = 0 GB.
WARNING 08-23 16:12:10 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 08-23 16:12:18 [config.py:717

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 08-23 16:12:21 [loader.py:458] Loading weights took 0.70 seconds
INFO 08-23 16:12:21 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 08-23 16:12:21 [model_runner.py:1140] Model loading took 1.1649 GiB and 1.371770 seconds
INFO 08-23 16:12:30 [worker.py:287] Memory profiling takes 8.50 seconds
INFO 08-23 16:12:30 [worker.py:287] the current vLLM instance can use total_gpu_memory (4.00GiB) x gpu_memory_utilization (0.56) = 2.23GiB
INFO 08-23 16:12:30 [worker.py:287] model weights take 1.16GiB; non_torch_memory takes 0.02GiB; PyTorch activation peak memory takes 0.70GiB; the rest of the memory reserved for KV Cache is 0.35GiB.
INFO 08-23 16:12:30 [executor_base.py:112] # cuda blocks: 203, # CPU blocks: 0
INFO 08-23 16:12:30 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 1.59x
INFO 08-23 16:12:30 [vllm_utils.py:671] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 08-23 16:12:30 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead

Capturing CUDA graph shapes:   0%|          | 0/19 [00:00<?, ?it/s]

INFO 08-23 16:12:42 [model_runner.py:1592] Graph capturing finished in 13 secs, took 0.24 GiB
INFO 08-23 16:12:42 [vllm_utils.py:678] Unsloth: Patched vLLM v0 graph capture finished in 13 secs.
INFO 08-23 16:12:43 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 21.83 seconds
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']
